In [1]:
import os
import sys
# from langchain_community.document_loaders import pypdf
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Weaviate
from langchain.agents import AgentType, initialize_agent
from langchain.schema import SystemMessage
from langchain.embeddings import OpenAIEmbeddings
from typing import List,Union
from weaviate.embedded import EmbeddedOptions
from dotenv import load_dotenv

import sys
sys.path.append('../utils')  # Adjust the path accordingly



In [2]:
from splitters import *
from retrivers import *

Choosing an appropriate chunking and text splitter mechanism is crutial for the performance of the RAG system

In [3]:
from langchain_community.document_loaders import PyPDFLoader
# Lets start by loading the document 
loader = PyPDFLoader('../data/RobinsonAdvisory.pdf')
contract = loader.load()

In [ ]:
%load_ext autoreload
%autoreload 2


In [4]:
recursive_doc = RecursiveSplitter(contract)

In [7]:
character_doc = characterSplitter(contract)

In [8]:
semantic_doc = SemanticSplitter(contract)

/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


In [9]:
qa_docx_file_path = "../data/Robinson Q&A.docx"


In [11]:
from langchain.vectorstores import Weaviate
import docx
from langchain_openai import ChatOpenAI

import weaviate
from dotenv import load_dotenv,find_dotenv
from weaviate.embedded import EmbeddedOptions

In [20]:
def create_vectorStore(doc):
    client = weaviate.Client(embedded_options=EmbeddedOptions)
    vectorstore = Weaviate.from_documents(client =  client,
                                        documents= doc,
                                        embedding=OpenAIEmbeddings(),
                                        by_text = False)
    return vectorstore

In [21]:
chara_vS = create_vectorStore(character_doc)
recur_vs = create_vectorStore(recursive_doc)
sem_vs = create_vectorStore(semantic_doc)

/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/weaviate/warnings.py:158: DeprecationWarning: Dep016: You are using the Weaviate v3 client, which is deprecated.
            Consider upgrading to the new and improved v4 client instead!
            See here for usage: https://weaviate.io/developers/weaviate/client-libraries/python
            
  warnings.warn(


embedded weaviate is already listening on port 8079


/home/kerod/Desktop/week_11/RAG-system-for-Contract-Q-A/week_11/lib/python3.10/site-packages/pydantic/main.py:1024: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


embedded weaviate is already listening on port 8079
embedded weaviate is already listening on port 8079


In [22]:
char_retriver = chara_vS.as_retriever()
recur_retriver = recur_vs.as_retriever()
sem_retriver = sem_vs.as_retriever()


In [23]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)
from langchain.schema.output_parser import StrOutputParser

In [26]:
def llm_chain(retriever):
    # Define LLM
    llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

    # Define prompt template
    prompt_template = """You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. 
    Question: {question} 
    Context: {context} 
    Answer:
    """

    prompt = ChatPromptTemplate.from_template(prompt_template)


    retrieval = RunnableParallel(
        {"context": retriever,  "question": RunnablePassthrough()} 
    )

    chain = retrieval | prompt | llm | StrOutputParser()

    return chain

In [27]:
char_chain = llm_chain(char_retriver)
recur_chain = llm_chain(recur_retriver)
sem_chain = llm_chain(sem_retriver)

In [29]:
from docx import Document
def extract_questions_answers(docx_file):
    questions = []
    answers = []

    doc = Document(docx_file)

    # Iterate through paragraphs
    for i in range(len(doc.paragraphs)):
        text = doc.paragraphs[i].text.strip()
        # Alternate paragraphs are questions and answers
        if text.startswith("Q"):
            question = text.split(": ", 1)[-1].strip()
            questions.append(question)
        elif text.startswith("A"):
            answer = text.split(": ", 1)[-1].strip()
            answers.append(answer)
            
            i += 1
        else:
            i += 1
        
        # if i % 2 == 0:
        #     questions.append(doc.paragraphs[i].text)
        # else:
        #     answers.append(doc.paragraphs[i].text)

    return questions, answers

questions, ground_truth = extract_questions_answers(qa_docx_file_path)


In [ ]:
def 

In [30]:
ground_truths = [[x] for x in ground_truth]

contexts = []
answers = []
for question in questions:
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(question)])
    answers.append(chain.invoke(question))


NameError: name 'retriever' is not defined

In [ ]:
from datasets import Dataset

# Preparing the dataset
data = {
            "question": questions,
            "answer": answers,
            "contexts": contexts,
            "ground_truth": ground_truth
        }

# Convert dict to dataset
dataset = Dataset.from_dict(data)

In [ ]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset=dataset, 
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

df = result.to_pandas()
